# Import Libraries

In [1]:
import requests # to get the html markdown from the url
from bs4 import BeautifulSoup # to make the html readable
import pandas as pd # to save table in a csv
from datetime import datetime # to append current time into file name


# Get the data from website

In [2]:
url_surplus = 'https://web.byui.edu/SurplusList/'
html_data = requests.get(url_surplus)
soup = BeautifulSoup(html_data.text, 'html.parser')


# Build Table

In [3]:
rows=[]
for item in soup.find_all('tr')[13:]:
    row={}
    row['Item Number'] = item.select('td')[0].text
    row['Qty'] = item.select('td')[1].text
    row['Description'] = item.select('td')[2].text
    row['Condition'] = item.select('td')[3].text
    row['Price'] = item.select('td')[4].text
    row['Location'] = item.select('td')[5].text
    row['Quick/Bid'] = item.select('td')[6].text
    row['Image'] = item.select('td')[7].text
    rows.append(row)


In [4]:
data = pd.DataFrame(rows)
data

,Item Number,Qty,Description,Condition,Price,Location,Quick/Bid,Image
0,75298,1,22 Oz Stainless Steel Top Water Bottle (Blue/P...,GOOD,$0.50,EA,Quick,Image
1,76631,15,4 in one mini locker units,GOOD,$15.00,EA,Quick,
2,75668,7,5' storage/file cabinet,GOOD,$5.00,EA,Quick,
3,73180,1,8-Pack Verbatim Brand Blank Logo CD-R CDR 80,POOR,$0.25,AMT,Quick,Image
4,75457,1,Acer Computer Charger,GOOD,$2.00,EA,Quick,Image
...,...,...,...,...,...,...,...,...
149,74697,1,White Neewer 2.4 GHz Wireless Remote in box or...,EXCELLENT,$1.25,EA,Quick,Image
150,75683,10,Wrestling Singlet,GOOD,$1.25,EA,Quick,Image
151,75682,14,Wrestling Singlet,GOOD,$1.25,EA,Quick,Image
152,75306,1,Zeroxposur Jacket for Man Size XXL (Black),GOOD,$1.75,EA,Quick,Image


## Save to CSV 

In [6]:
date = datetime.now().strftime("%d-%m-%Y %H%M%S") # get local time as string

filePath = "../Surplus Sale Items - Historical Data/SurplusSaleItems " + date + " - Webscrape.csv"

data.to_csv(filePath, index=False) # save to file path

## Simple Data Analysis 